# Playground

My playground, for learning / debugging / visualizing purposes

## Look up dataset images: in a simple way

In [ ]:
import os
from os import path as osp
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import ipywidgets as widgets

In [2]:
data_path = "./data/"
dataset_path = osp.join(data_path, 'CVACT_full', 'streetview')
dataset_filenames = sorted(os.listdir(dataset_path))

In [3]:
def show_img(dataset_path, filenames, idx):
    img_path = osp.join(dataset_path, filenames[idx])
    img = np.asarray(Image.open(img_path))
    print(f'\tFile path: {img_path}')
    print(f'\tImage shape: {img.shape}')
    plt.imshow(img)
    plt.show()

def look_dataset(dataset_path, filenames, s=0, l=100):
    print(f'Dataset path: {dataset_path}')
    print(f'Number of images in {dataset_path}: {len(dataset_filenames)}')
    print(f'Current visualizing range: {s} - {s + l}')
    widgets.interact(lambda idx: show_img(dataset_path, filenames, idx),
                     idx=widgets.IntSlider(min=s, max=min(len(dataset_filenames) - 1, s + l), step=1))

In [4]:
look_dataset(dataset_path, dataset_filenames, 300, 100)

Dataset path: ./data/CVACT_full/streetview
Number of images in ./data/CVACT_full/streetview: 128331
Current visualizing range: 300 - 400


interactive(children=(IntSlider(value=300, description='idx', max=400, min=300), Output()), _dom_classes=('wid…

## Look up CVACT dataset: interactively

In [1]:
from pathlib import Path
from orissl_cvm.datasets.cvact_dataset import CVACTDataset
from orissl_cvm.utils.tools import input_transform
from torch.utils.data import DataLoader

In [76]:
root_dir = Path('./data/CVACT_full/').absolute()

train_dataset = CVACTDataset(root_dir, 
                             mode='train', 
                             nNeg=3, 
                             transform=input_transform(),
                             bs=32, 
                             threads=6, 
                             margin=0.1)

Construct neighbor searches: kd_tree
Finding positive neighbors for 8646 queries
Finding non-negative neighbors for 8646 queries


In [77]:
# print(train_dataset.qIdx)
# print(train_dataset.pIdx)
# print(train_dataset.nonNegIdx)
# print(train_dataset.posDistThr)
# print(train_dataset.negDistThr)


In [78]:
from orissl_cvm.models.siamese import GoodNet
import torch

config = {
    'global_params': {'pooling': 'max'}
}
checkpoint = torch.load("work_dirs/Apr05_12-36-10_cvact_resnet50/checkpoints/checkpoint.pth.tar", map_location=lambda storage, loc: storage)
model = GoodNet(config['global_params'])
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [79]:
# divides dataset into smaller cache sets
train_dataset.new_epoch()

# creates triplets on the smaller cache set
# train_dataset.update_subcache()
train_dataset.update_subcache(model, 512)

print(len(train_dataset.triplets))
print(train_dataset.triplets)

>> Searching for hard negatives...
961
[([4334, 5575, 3114, 6239], [-1, 0, 0, 0]), ([5868, 5575, 3114, 6239], [-1, 0, 0, 0]), ([2844, 5575, 3114, 6239], [-1, 0, 0, 0]), ([3685, 5575, 3114, 6239], [-1, 0, 0, 0]), ([3199, 5575, 3114, 6239], [-1, 0, 0, 0]), ([1869, 5575, 3114, 6239], [-1, 0, 0, 0]), ([425, 5575, 3114, 6239], [-1, 0, 0, 0]), ([7778, 5575, 3114, 6239], [-1, 0, 0, 0]), ([329, 5575, 3114, 6239], [-1, 0, 0, 0]), ([8258, 5575, 3114, 6239], [-1, 0, 0, 0]), ([3779, 5575, 3114, 6239], [-1, 0, 0, 0]), ([5482, 5575, 3114, 6239], [-1, 0, 0, 0]), ([4038, 5575, 3114, 6239], [-1, 0, 0, 0]), ([3433, 5575, 3114, 6239], [-1, 0, 0, 0]), ([7565, 5575, 3114, 6239], [-1, 0, 0, 0]), ([1653, 5575, 3114, 6239], [-1, 0, 0, 0]), ([1554, 5575, 3114, 6239], [-1, 0, 0, 0]), ([33, 5575, 3114, 6239], [-1, 0, 0, 0]), ([2418, 5575, 3114, 6239], [-1, 0, 0, 0]), ([8091, 5575, 3114, 6239], [-1, 0, 0, 0]), ([2377, 5575, 3114, 6239], [-1, 0, 0, 0]), ([1146, 5575, 3114, 6239], [-1, 0, 0, 0]), ([6808, 5575, 3114

In [80]:
# create data loader
opt = {'batch_size': 16, 'shuffle': True, 'collate_fn': CVACTDataset.collate_fn}
training_loader = DataLoader(train_dataset, **opt)

In [81]:
import matplotlib.pyplot as plt
import numpy as np

def denormalize(im):
	image = im.numpy()
	im = (image - np.min(image)) / (np.max(image) - np.min(image))
	im = np.ascontiguousarray(im * 255, dtype=np.uint8)
	return im

def visualize_triplet(batch, sample_idx):

	query, negatives, meta = batch
	negCounts, indices, keys = meta['negCounts'], meta['indices'], meta['keys']

	B = query[0].shape[0]

	num_ns = negCounts[sample_idx].item()
	num_qns = num_ns + 1

	neg_start = 0
	start = 0
	if sample_idx > 0: 
		neg_start = negCounts[:sample_idx].sum().item()
		start = neg_start + sample_idx * 1
	# print(sample_idx, start)

	fig, axes = plt.subplots(nrows=num_qns, ncols=2, figsize=(15,9))
	fig.suptitle(
		f'Navigate dataloader of CVACT: current batch, sample {sample_idx} (1 query and {num_ns} negatives)',
		fontsize=15)
	fig.tight_layout()
	fig.subplots_adjust(top=0.9)
	
	axes[0,0].imshow(np.transpose(denormalize(query[0][sample_idx]),(1,2,0)))
	axes[0,0].set_title(
		f"Query ==> ground image\nidx: {indices[start]}, file name: {keys[sample_idx]['query']['gr_img']}")

	axes[0,1].imshow(np.transpose(denormalize(query[1][sample_idx]),(1,2,0)))
	axes[0,1].set_title(
		f"Query ==> satellite image\nidx: {indices[start]}, file name: {keys[sample_idx]['query']['sa_img']}")

	# axes[1,0].imshow(np.transpose(denormalize(positive[0][sample_idx]),(1,2,0)))
	# axes[1,0].set_title(
	# 	f"Positive ==> ground image\n{keys[sample_idx]['positive']['gr_img']}")
	
	# axes[1,1].imshow(np.transpose(denormalize(positive[1][sample_idx]),(1,2,0)))
	# axes[1,1].set_title(
	# 	f"Positive ==> satellite image\n{keys[sample_idx]['positive']['sa_img']}")

	for i in range(num_ns):
		axes[1+i,0].imshow(np.transpose(denormalize(negatives[0][neg_start+i]),(1,2,0)))
		axes[1+i,0].set_title(
			f"Negative {i} ==> ground image\nidx: {indices[start+i+1]}, file name: {keys[sample_idx]['negatives'][i]['gr_img']}")

		axes[1+i,1].imshow(np.transpose(denormalize(negatives[1][neg_start+i]),(1,2,0)))
		axes[1+i,1].set_title(
			f"Negative {i} ==> satellite image\nidx: {indices[start+i+1]}, file name: {keys[sample_idx]['negatives'][i]['sa_img']}")

	plt.show()

In [82]:
# visualize triplets

import ipywidgets as widgets
from IPython.display import display, clear_output

button = widgets.Button(
    description='Next Batch',
    layout=widgets.Layout(width='10%')
)
out = widgets.Output()
bs = training_loader.batch_size
it = iter(training_loader)

def on_button_clicked(_):
    # "linking function with output"
    with out:
        try:
            batch = next(it)
        except StopIteration:
            print("Data loader ran out.")
        clear_output()
        # display(f'')
        sample_slider = widgets.IntSlider(
            value=0, min=0, max=bs-1, step=1, 
            description='Sample:',
            layout=widgets.Layout(width='25%')
        )
        widgets.interact(lambda sample_idx: visualize_triplet(batch, sample_idx),
                         sample_idx=sample_slider)
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button,out])
    

## Look up VGG16 architecture

In [1]:
import torchvision.models as models
import torch.nn as nn

In [2]:
enc = models.vgg16(pretrained=True)
print(enc)

In [ ]:
layers = list(enc.features.children())

In [21]:
print(len(layers))
for layer in layers[:-5]:
    print(layer)
    for p in layer.parameters():
        pass

## Create a CVACT-mini dataset info

手动取原list的前1/10作为我们mini集的数据

```cat trainlist.txt | wc -l```

"86469"

```cat trainlist.txt | head -n 8646 > ../CVACT_infos_mini/trainlist.txt```

Same for vallist.txt, testlist.txt

## Others

### zip

In [31]:
import numpy as np

a1 = [np.zeros(1)]
a2 = a1 * 2
b1 = [np.ones(1)]
b2 = b1 * 2

print(a2)
print(b2)

[array([0.]), array([0.])]
[array([1.]), array([1.])]


In [35]:
c = 'token3'
d = 'token4'
for x, y in zip([a2, b2], [c, d]):
    print(x, y)
for x, y in zip([a1, a2, b1, b2], [c, d]):
    print(x, y)
# 直观来说，zip起来直接一起iterate更方便

[array([0.]), array([0.])] token3
[array([1.]), array([1.])] token4
[array([0.])] token3
[array([0.]), array([0.])] token4


In [36]:
uppercase = ['A', 'B', 'C']
lowercase = ['a', 'b', 'c']
numbers = [1, 2, 3]

for x, y, z in zip(uppercase, lowercase, numbers):
    print(x, y, z)

A a 1
B b 2
C c 3
